# <center> Advanced ML pipeline with segmentation_models and Callbacks
    
---

## In this Session
* Understand the dataset
* Create the dataset preprocessing pipeline
* Understand each step in preprocessing pipeline

# Importing libraries

In [ ]:
from matplotlib import pyplot as plt  # Displaying images
from skimage.io import imread         # Read the images
import numpy as np                    # Data Handling
import datetime                       # Used in Naming
import math                           # Math operations
import os                             # Directory files

# One Hot Encoded Mask and Dataset Building
from tensorflow.keras.utils import to_categorical, Sequence
import tensorflow as tf
import keras

# warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set the root directory paths for images and masks
img_dir = '/kaggle/input/images/render'
mask_dir = '/kaggle/input/images/clean'

# Sort the files in root directories, Create and Store the complete image and mask paths
images = [os.path.join(img_dir, x) for x in sorted(os.listdir(img_dir))]
masks = [os.path.join(mask_dir, x) for x in sorted(os.listdir(mask_dir))]


# First 8000 images to be used for training
X_train = images[:8000]
y_train = masks[:8000]

# Remaining can be used for validation purpose
X_valid = images[8000:-4]
y_valid = masks[8000:-4]

# Save some for testing purpose (last 4)
X_test = images[-4:]
y_test = masks[-4:]

### Break Down of Mask Preprocessing


In [ ]:
# Consider an example in training data to understand this
mask_sample_path = y_train[1]
print(mask_sample_path)

In [ ]:
# Read the mask as gray scale
sample_mask_arr = imread(mask_sample_path, as_gray=True)
print(sample_mask_arr.shape)

In [ ]:
# Cropping the mask
sample_mask_cropped = sample_mask_arr[:480, :480]
print(sample_mask_cropped.shape)

In [ ]:
# Mask with original size and after cropping
fig, axes = plt.subplots(1, 2, figsize=(8, 4))

plot_data = ((sample_mask_arr, sample_mask_cropped),
            (sample_mask_arr.shape, sample_mask_cropped.shape))

for (ax, arr, title) in zip(axes,*(plot_data)) :
    ax.imshow(arr)
    ax.set_title(title)


In [ ]:
# Check the unique values in cropped_mask
np.unique(sample_mask_cropped)

In [ ]:
# What happens if you divide them with 0.07?
np.unique(sample_mask_cropped)//0.07

In [ ]:
# We need to convert the values of 3 to 2 and 10 to 3 after floor operation with 0.07
adjusted_mask = sample_mask_cropped//0.07
adjusted_mask[adjusted_mask == 3.0] = 2.0
adjusted_mask[adjusted_mask == 10.0] = 3.0
print(np.unique(adjusted_mask))

In [ ]:
# Check if any changes in the output (the colors are a bit different! as pixel values changed)
plt.imshow(adjusted_mask)

In [ ]:
# Get 4 channel one hot encoded mask
final_sample_mask = to_categorical(adjusted_mask,num_classes=4)
final_sample_mask

### Excercise to get from one hot to integer encoded mask (post processing)

In [ ]:
final_sample_mask.shape

In [ ]:
int_encoded_mask = np.argmax(final_sample_mask, axis=-1)
int_encoded_mask

In [ ]:
plt.imshow(int_encoded_mask)

In [ ]:
# A dummy 4 channel array
np.random.seed(42)
dum_arr = np.random.rand(3, 3, 4)
dum_arr # 1, 3, 3, # 0, 1, 0 # 1, 2, 1

# [1, 3, 3]
# [0, 1, 0]
# [1, 2, 2]

In [ ]:
# First row has max value of 0.95 on index 1, hence, 1 is returned on (0,0)
# Similarly, second row has max value of 0.86 on index 3, hence 3 is returned for (0,1)
# So on ...
np.argmax(dum_arr, axis=-1)

## Custom Data Generator to prepare the Dataset

In [ ]:
# Dataset Pipeline
class LunarDataset(Sequence):

    # Constructor - x_set, y_set, batch_size, dims, classes
    def __init__(self, x_set, y_set, batch_size, dims, classes):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.img_height, self.img_width = dims
        self.classes = classes

    # Number of Batches --> total length of images / batch size --> Ceil operation
    def __len__(self):
        return math.ceil(len(self.x)/self.batch_size)

    # Fetch the data in batches by using iter and next opertions
    def __getitem__(self, idx):
        # Get start and end indexes to create a batch of batch size
        start_index = idx * self.batch_size
        end_index = (idx + 1) * self.batch_size
        '''
        0*2 : 1*2 --> 0:2 --> 0, 1
        1*2 : 2*2 --> 2:4 --> 2, 3
        2*2 : 3*2 --> 4:6 --> 4, 5
        '''

        # Prepare X and y batches
        batch_x = self.x[start_index : end_index]
        batch_y = self.y[start_index : end_index]

        # Empty lists to append preprocessed Images and Masks Array from the for loop
        xtr = []
        ytr = []

        # For every  image and mask in one batch do the following preprocessing
        for idx, (filename_x, filename_y) in enumerate(zip(batch_x, batch_y)):

            # Image preprocessing
            img = imread(filename_x)[:self.img_height, :self.img_width, :]/255.0
            img = img.astype(np.float32)
            xtr.append(img)

            # Mask preprocessing
            mask = imread(filename_y, as_gray=True)[:self.img_height, :self.img_width]//0.07 # 0, 1, 3, 10
            mask[mask == 3] = 2
            mask[mask == 10] = 3
            mask = to_categorical(mask, num_classes = self.classes)
            ytr.append(mask)

        # Convert list to arrays ensuring the dtype of mask is also float32
        xtr = np.array(xtr)
        ytr = np.array(ytr).astype(np.float32)

        # Return the preprocessed batch of images and respective mask as output
        return xtr, ytr

In [ ]:
# Parameters
batch_size = 16
dims = (480, 480)
num_classes = 4

# Dataset Creation
train_dataset = LunarDataset(X_train, y_train, batch_size, dims, num_classes)
valid_dataset = LunarDataset(X_valid, y_valid, batch_size, dims, num_classes)

## Let's visualize our masks

In [ ]:
# Taking a sample batch from train_dataset
batch = next(iter(train_dataset)) # Batch Size, Height, Width, Channels (Images, Masks)

# Check the shape of batch created --> Images and Masks
print(batch[0].shape) # 16 images in a batch
print(batch[1].shape) # 16 respective masks in a batch

In [ ]:
# Get the mask
sample = batch[1][1] # Second in the batch

In [ ]:
# Creating Subplot for better visualization
fig, ((a1, a2, a3), (a4, a5, a6)) = plt.subplots(2, 3, figsize = (10, 8))

# For different axes and titles
for i, (ax,title) in enumerate(zip((a1, a2, a3, a4, a5, a6),
                                   ('Original', 'Combined Mask', 'Background', 'Large Rocks', 'Sky', 'Small Rocks'))):
    if i == 0:
        ax.imshow(batch[0][1])                    # Second Image in the batch (Original)
    elif i == 1:
        ax.imshow(np.argmax(sample, axis=-1))     # Converts One Hot encoded mask to Integer Encoded Mask (single channel)
    else:
        ax.imshow(sample[:, :, i-2])              # Channel Wise Output from Mask

    # Set Title and turn off the axis
    ax.set_title(title, fontsize=15, weight='bold')
    ax.axis('off')

# Adjust Layout and Display the Subplot
plt.tight_layout()
plt.show()

### Let's check out some basic steps of transfer learning using a pretrained model (VGG16)


## segmentation_models

**segmentation_models** is a python library with Neural Networks for Image Segmentation based on Keras and TensorFlow.

The main features of this library are:

* High level API (just two lines of code to create model for segmentation)
* 4 models architectures for binary and multi-class image segmentation (including legendary Unet)
* 25 available backbones for each architecture
* All backbones have pre-trained weights for faster and better convergence
* Helpful segmentation losses (Jaccard, Dice, Focal) and metrics (IoU, F-score)

In [ ]:
# run this command to directly install the library in our notebook

!pip install segmentation_models

* Provide environment variable SM_FRAMEWORK=keras / SM_FRAMEWORK=tf.keras before import segmentation_models
* Change framework sm.set_framework('keras') / sm.set_framework('tf.keras')

In [ ]:
# By default it tries to import keras, if it is not installed, it will try to start with tensorflow.keras framework
os.environ["SM_FRAMEWORK"] = "tf.keras" # Set the environment variable SM_FRAMEWORK to "tf.keras"
import segmentation_models as sm        # It will import without any errors if env variable is set properly
sm.set_framework('tf.keras')            # Use segmentation_models library and set the framework to TensorFlow's Keras

In [ ]:
# Set the last axis of the tensor as channel axis
tf.keras.backend.set_image_data_format('channels_last')
# Explicitly setting this is not always necessary, as 'channels_last' is the default setting

## Building our UNet model with segmentation_models

In [ ]:
# Parameters for UNET using Segmentation Models
BACKBONE = 'vgg16'
input_shape = (480, 480, 3)
n_classes = 4
activation = 'softmax'


# Even though backbone doesn't have encoder-decoder structure, it is still able to build it thanks to sm library
model = sm.Unet(backbone_name = BACKBONE,
               input_shape = input_shape,
               classes = n_classes,
               activation = activation,)
#                 encoder_weights = 'imagenet',
#                 encoder_freeze = True)

# Check all the different parameters of UNET, try with them
# Uncomment the above two lines of UNET for transfer learning

# Check the model summary
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, 'unet_vgg16.png')

## Compile the model

#### IOU SCORE

* The IoU score will be high if there is much overlap between the anticipated and ground truth boxes.
* In contrast, a low overlap will result in a low IoU score.
* An IoU score of 1 indicates a perfect match between the projected box and the ground truth box.
* Whereas a score of 0 means no overlapping between the boxes.
  
<img src='https://b2633864.smushcdn.com/2633864/wp-content/uploads/2016/09/iou_equation.png?lossy=2&strip=1&webp=1' width = 50%>

In [ ]:
""" Hyperparameters """
lr = 1e-4
batch_size = 16
epochs = 4

# metrics for result validation
metrics = [sm.metrics.IOUScore(threshold=0.5)]

# compiling the model --> Try changing the loss function to jacard_loss from sm library and see changes!
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(lr),
              metrics = metrics)


""" Callbacks """
callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor = 'val_iou_score',
            mode = 'max',
            patience = 2,
            factor = 0.1,
            verbose = 1,
            min_lr = 1e-6
        ),

        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_iou_score',
            patience = 3,
            mode = 'max',
            verbose = 1
        )
    ]


- model is compiled with **loss**="categorical_crossentropy",  **optimizer**=Adam, **metrics**=iou_score

**Callbacks** is a tool to customize the behavior of a Keras model during training, evaluation, or inference.

**ReduceLROnPlateau:** Reduce learning rate when a metric has stopped improving.

**EarlyStopping:** Stop training when a monitored metric has stopped improving.

## Train the Model

In [ ]:
# Fitting the model
model_history = model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        callbacks=callbacks,
    )

## Generate Prediction
* We shall also use this code in our API to build our Streamlit webapp using FastAPI.

In [ ]:
# function to predict result
def predict_image(img_path, mask_path, model):
    ## Basic Setup
    H = 480
    W = 480
    num_classes = 4

    ## Read Images and Preprocess
    img = imread(img_path)
    img = img[:W, :H]
    img = img / 255.0
    img = img.astype(np.float32)

    ## Read mask and Preprocess
    mask = imread(mask_path, as_gray = True)
    mask = mask[:W, :H]

    ## Prediction mask and Postprocess
    pred_mask = model.predict(np.expand_dims(img, axis=0))
    pred_mask = np.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[0]


    ## Calculating IOU score
    inter = np.logical_and(mask, pred_mask)
    union = np.logical_or(mask, pred_mask)

    iou = inter.sum()/union.sum()

    return img, mask, pred_mask, iou

In [ ]:
X_test[0], y_test[0]

In [ ]:
index_test = 0
img_path = X_test[index_test]
mask_path = y_test[index_test]

img, mask, pred_mask, iou = predict_image(img_path, mask_path, model)

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (15, 10))

ax1.set_title("Input Image")
ax1.imshow(img)

ax2.set_title("True Mask")
ax2.imshow(mask)

ax3.set_title("Predicted mask with IOU score %.2f"%(iou))
ax3.imshow(pred_mask)

plt.show()

## Save the Model
* Once satisfied with the model performance, you can save the model.

In [ ]:
from tensorflow.keras.models import load_model, save_model

In [ ]:
# Save the Model
save_model(model, 'LunarModel.h5')

## Test the saved model

In [ ]:
# Load the Model and test it
loaded_model = load_model('/kaggle/working/LunarModel.h5')
_, _, _, iou_score = predict_image(img_path, mask_path, loaded_model)
print(iou_score)

## THE END